In [26]:

import pandas as pd
df = pd.read_csv("ICD9_codes_deconcat.csv")
df2 = pd.read_csv("dataset/PROCEDURES_ICD.csv.gz")
indice = pd.read_csv("dataset/D_ICD_PROCEDURES.csv.gz")

df.head()

,Unnamed: 0,SUBJECT_ID,HADM_ID,ICD9_CODE_procedures,ICD9_CODE_procedures_preprocess_aux
0,0,23,152223,3613,3613.0
1,1,23,152223,3615,3615.0
2,2,23,152223,3961,3961.0
3,3,23,152223,8872,8872.0
4,4,23,152223,9904,9904.0


In [52]:


# libraries
from cucco import Cucco
import ftfy
import pandas as pd
import numpy as np
import re
import plotly.express as px
#import spacy
import itertools

from nltk.probability import FreqDist
import gensim


import nltk; nltk.download('stopwords')

#nlp = spacy.load('es_core_news_sm')

# preprocessing class
class pre_procesamientotw:
  def __init__(self,text,stopword_list):
    self.text = text
    self.stopword_list = stopword_list
    
  def proceso_español(self,text):
    try:
       text = text.lower()
    except:
        pass

    text = re.sub("@[A-Za-z0-9_]+","", text) # remove mentions
    text = re.sub("#[A-Za-z0-9_]+","", text) # remove hashtags
    text = re.sub(r"http\S+", "", text) # remove http urls
    text = re.sub(r"www.\S+", "", text) # remove www urls
    text = re.sub('\S+@\S+', "", text) # remove emails

    # remove accents
    a,b = 'áéíóúüñÑÁÉÍÓÚÜ','aeiouunnaeiouu' 
    trans = str.maketrans(a, b)
    text = text.translate(trans)

    # replace punctuation, extra whitespaces
    normEsp = Cucco()
    norms = ['remove_stop_words', 'replace_punctuation', 'remove_extra_whitespaces']
    
    
    text = normEsp.normalize(text, norms)
    text = ftfy.fix_encoding(text)

    return text
  
    def proceso_español(self,text):
      try:
         text = text.lower()
      except:
          pass

      text = re.sub("@[A-Za-z0-9_]+","", text) # remove mentions
      text = re.sub("#[A-Za-z0-9_]+","", text) # remove hashtags
      text = re.sub(r"http\S+", "", text) # remove http urls
      text = re.sub(r"www.\S+", "", text) # remove www urls
      text = re.sub('\S+@\S+', "", text) # remove emails

      # remove accents
      a,b = 'áéíóúüñÑÁÉÍÓÚÜ','aeiouunnaeiouu' 
      trans = str.maketrans(a, b)
      text = text.translate(trans)

      normEsp = Cucco()
      norms = ['remove_stop_words', 'replace_punctuation', 'remove_extra_whitespaces']

      text = normEsp.normalize(text, norms)
      text = ftfy.fix_encoding(text)

      return text

  def str_stem(self,s,stopword_list):

      s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) 
      try:
         s = s.lower()
      except:
          pass

      s = s.replace("  "," ") #remove double space
      #remove special character and split number
      s = s.replace(",","") #could be number / segment later
      s = s.replace("$"," ")
      s = s.replace("?"," ")
      s = s.replace("-"," ")
      s = s.replace("//","/")
      s = s.replace("..",".")
      s = s.replace(" / "," ")
      s = s.replace(" \\ "," ")
      s = s.replace("."," . ")
      s = re.sub(r"(^\.|/)", r"", s)
      s = re.sub(r"(\.|/)$", r"", s)
      s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
      s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
      s = s.replace(" x "," xbi ")
      s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
      s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)
      s = s.replace("*"," xbi ")
      s = s.replace(" by "," xbi ")
      #convert mesurement unit to standard form.
      s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
      s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
      s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
      s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
      s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
      s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
      s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
      s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
      s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
      s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
      s = s.replace("°"," degrees ")
      s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
      s = s.replace(" v "," volts ")
      s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
      s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
      s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
      s = re.sub("@[A-Za-z0-9_]+","", s) # remove mentions
      s = re.sub("#[A-Za-z0-9_]+","", s) # remove hashtags
      s = re.sub(r"http\S+", "", s) # remove http urls
      s = re.sub(r"www.\S+", "", s) # remove www urls
      s = re.sub('[()!?]', ' ', s)
      s = re.sub('\[.*?\]',' ', s)
     # doc = nlp(s)
     # s = [tok.lemma_.lower() for tok in doc]
      s = " ".join(s) 
      s = (" ").join([z for z in s.split(" ") if z not in stopword_list])
      s = s.lower()

      return s

  def sent_to_words(self,sentences):

      ps = []
      for sentence in sentences:
          #simple preprocess from gensim
          ps.append( gensim.utils.simple_preprocess(str(sentence), deacc=True) )
 
      ps = [item for sublist in ps for item in sublist]

      return ' '.join(ps)
   

  def normalize(self,text,stopword_list):
      # considere words with 4 words or more
      lexical_tokens = [t.lower() for t in text.split(' ') if len(t) > 4]
      #removes stop words
      lexical_tokens = [i for i in lexical_tokens if i not in stopword_list]
      return ' '.join(lexical_tokens)

  def process_esp(self,s):
      clean = s.apply( self.proceso_español)
      return clean
    
  
  def stem(self,s,stopword_list):
      clean = s.apply(lambda x: self.str_stem(x,stopword_list))
      return clean
  
  def sent_to_words_a(self,text):
      text_ = self.text.apply(lambda x: x.split(' '))
      clean = text_.apply(self.sent_to_words)
      return clean
  
    
  def normalize_a(self,s,stopword_list):
      clean = s.apply(lambda x: self.normalize(x,stopword_list))
      return clean
 
  def lista(self,flat_list):
     # obtain the frequency of the words 
     #returns a dataframe with words and frequency
      fdist=FreqDist(flat_list)
      dic1 = dict(fdist)
      dict_l = pd.DataFrame(list(dic1.items()),
                          columns=['Letra', 'Freq'])
      table = pd.pivot_table(dict_l, values=[ 'Freq'],  index=['Letra'],
                        aggfunc={'Freq': np.sum })
      table =table.sort_values(by = "Freq", ascending=False)  
      
      table['Letra_'] = table.index
      
      return table
    
      # applys to text the 4 preprocessing functions
  def pre_process(self,text,stopword_list):
      
      t = self.process_esp( self.text )
      t = self.stem( t, stopword_list)
      t = self.sent_to_words_a( t )
      t = self.normalize_a( t,stopword_list )

      return t
    


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cgarciay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [53]:
indice.dtypes


ROW_ID          int64
ICD9_CODE       int64
SHORT_TITLE    object
LONG_TITLE     object
dtype: object

In [29]:
#df["ICD9_CODE"] = df["ICD9_CODE"].astype(int)
df.dtypes


Unnamed: 0                               int64
SUBJECT_ID                               int64
HADM_ID                                  int64
ICD9_CODE_procedures                     int64
ICD9_CODE_procedures_preprocess_aux    float64
dtype: object

In [30]:
df =  df.rename(columns={"ICD9_CODE_procedures":"ICD9_CODE"})

merge_input = pd.merge(df, indice, on=["ICD9_CODE",], how='left')
merge_input.head()

,Unnamed: 0,SUBJECT_ID,HADM_ID,ICD9_CODE,ICD9_CODE_procedures_preprocess_aux,ROW_ID,SHORT_TITLE,LONG_TITLE
0,0,23,152223,3613,3613.0,663.0,Aortocor bypas-3 cor art,(Aorto)coronary bypass of three coronary arteries
1,1,23,152223,3615,3615.0,665.0,1 int mam-cor art bypass,Single internal mammary-coronary artery bypass
2,2,23,152223,3961,3961.0,759.0,Extracorporeal circulat,Extracorporeal circulation auxiliary to open h...
3,3,23,152223,8872,8872.0,3233.0,Dx ultrasound-heart,Diagnostic ultrasound of heart
4,4,23,152223,9904,9904.0,3846.0,Packed cell transfusion,Transfusion of packed cells


In [ ]:
stopword_list = stopwords.words('english') 

p =   pre_procesamientotw(data_op['text'],stopword_list)
clean = p.pre_process(data_op['text'],stopword_list)

data_op['clean_text'] = clean


In [ ]:
threshold.shape

In [13]:
len_ic9codes = [len(i)  for i in df["ICD9_CODE_procedures"]]
min(len_ic9codes)


1

In [6]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt


In [206]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model = AutoModel.from_pretrained("medicalai/ClinicalBERT")






In [8]:
list(tokenizer.vocab.keys())[5000:5020]

['##toire',
 '##sion',
 '##פ',
 'Genoa',
 '##continent',
 'view',
 '##ŋ',
 'ớ',
 'recognition',
 'giants',
 'ロ',
 'dispatched',
 '##zilla',
 'phrases',
 'principle',
 'darts',
 '##氏',
 'valleys',
 'indicating',
 'alike']

In [59]:

merge_input["LONG_TITLE"] = merge_input["LONG_TITLE"].astype(str)

In [58]:
merge_input["LONG_TITLE"]

0        (Aorto)coronary bypass of three coronary arteries
1           Single internal mammary-coronary artery bypass
2        Extracorporeal circulation auxiliary to open h...
3                           Diagnostic ultrasound of heart
4                              Transfusion of packed cells
                               ...                        
84325      Central venous catheter placement with guidance
84326    Continuous invasive mechanical ventilation for...
84327     Venous catheterization, not elsewhere classified
84328                       Insertion of endotracheal tube
84329    Enteral infusion of concentrated nutritional s...
Name: LONG_TITLE, Length: 84330, dtype: object

In [60]:
from nltk.corpus import stopwords
stopword_list = stopwords.words('spanish') 
p =   pre_procesamientotw(merge_input["LONG_TITLE"],stopword_list)
clean = p.pre_process(merge_input["LONG_TITLE"],stopword_list)

#data_op['clean_text'] = clean


In [61]:
merge_input["clean_codes"] = clean
merge_input.to_csv("clean/procedures_ic9.csv")



In [173]:
clean

0            aorto coronary bypass three coronary arteries
1           single internal mammary coronary artery bypass
2        extracorporeal circulation auxiliary heart sur...
3                              diagnostic ultrasound heart
4                                 transfusion packed cells
                               ...                        
84325           central venous catheter placement guidance
84326    continuous invasive mechanical ventilation con...
84327          venous catheterization elsewhere classified
84328                               insertion endotracheal
84329    enteral infusion concentrated nutritional subs...
Name: LONG_TITLE, Length: 84330, dtype: object

# Tokenization

In [191]:
text = clean[0]
text

'aorto coronary bypass three coronary arteries'

In [214]:
len(tokenized_text)

13

In [235]:
def embedding_(text,operation):
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():

        outputs = model(tokens_tensor, segments_tensors,)

        # Evaluating the model will return a different number of objects based on 
        # how it's  configured in the `from_pretrained` call earlier. In this case, 
        # becase we set `output_hidden_states = True`, the third item will be the 
        # hidden states from all layers. See the documentation for more details:
        # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
        hidden_states = outputs
        
    # Put the model in "evaluation" mode, meaning feed-forward operation.
    model.eval()
    if operation=="mean":
       sentence_embedding = torch.mean(hidden_states[0], dim=1)
       print(sentence_embedding.shape)
    elif operation =="sum":   
       sentence_embedding = torch.sum(hidden_states[0], dim=1)
       #print(sentence_embedding.shape)
       
    return sentence_embedding.numpy()



In [252]:
merge_input.shape

(84330, 9)

In [258]:
from tqdm import tqdm

In [263]:
concatenated_array = np.zeros((0,768))
for text in tqdm(clean):
    sentence_embedding = embedding_(text,"mean")
    
    concatenated_array = np.concatenate((concatenated_array,sentence_embedding), axis=0)  # Change the axis as needed
concatenated_array.shape

  0%|          | 4/84330 [00:00<1:09:09, 20.32it/s]

torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])


  0%|          | 12/84330 [00:00<55:08, 25.49it/s] 

torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])


  0%|          | 19/84330 [00:00<49:24, 28.44it/s]

...

torch.Size([1, 768])


 38%|███▊      | 32020/84330 [58:11<4:50:53,  3.00it/s]

torch.Size([1, 768])


 38%|███▊      | 32021/84330 [58:12<6:47:59,  2.14it/s]

torch.Size([1, 768])


# concatener par formar mis embeeding por admission_if,ham_id

In [241]:
concatenated_array.shape

(768,)

In [ ]:
df_promedio = df.groupby(['HAMD_ID', 'SUBJECT_ID'])[columnas_de_interes].mean().reset_index()


In [ ]:
demo = pd.read_csv("/Users/cgarciay/Desktop/Laval_Master_Computer/research/input_model_pred/sin_codigo.csv")

Current dimensions:

[# layers, # batches, # tokens, # features]

Desired dimensions:

[# tokens, # layers, # features]

Luckily, PyTorch includes the permute function for easily rearranging the dimensions of a tensor.

However, the first dimension is currently a Python list!

In [142]:
tokens_tensor.shape

torch.Size([1, 50])